In [1]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.chains import  LLMChain
from langchain.prompts import PromptTemplate
# from langchain.agents import TextGenerator
#from langchain.llms import TextGen


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np


In [2]:
def cosine_similarity(vec1, vec2):
    # Compute the dot product of vec1 and vec2
    dot_product = np.dot(vec1, vec2)
    
    # Compute the L2 norms (or magnitudes) of vec1 and vec2
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Compute the cosine similarity
    cos_sim = dot_product / (norm_vec1 * norm_vec2)
    
    return cos_sim

### Load data

In [3]:
## Convert raw markdown into "document" as prescribed by langchain
loader = TextLoader("./promo.md")
data = loader.load()


In [3]:
#"youtube_transcript_whitney.pdf"
# estimating_demand_with_Machine_learning
loader = PyPDFLoader("median_price_vox.pdf")
data = loader.load_and_split()

In [15]:
data[0].page_content

"#\ntactiq.io\nfree\nyoutube\ntranscript\n#\nWhy\nbuying\na\nhouse\nin\nthe\nUS\nis\nso\nhard\nright\nnow\n#\nhttps://www.youtube.com/watch/Exza1UYxXXM\n00:00:01.209\nWhen\nyou\nbuy\na\nhome\nin\nthe\nUS,\n00:00:02.877\nthere\nis\na\nbasic\npromise:\n00:00:04.587\nthat\nhome\nvalues\nwill\ngo\nup\nover\ntime.\n00:00:08.383\nAnd\nthat\nis\nwhat\nyou're\nseeing\non\nthis\nchart.\n00:00:11.433\nLet’s\nadjust\nit\nfor\ninflation...\n00:00:13.555\nok...\n00:00:14.431\njerkier,\nbut\nstill\nbasically\ngoes\nup.\n00:00:16.683\nThe\nstory\ngoes\nlike\nthis:\n00:00:18.351\nMoney,\nby\nitself,\nloses\nvalue\nover\ntime\n-that's\ninflation.\n00:00:22.647\nBut\nif\nyou\nown\nyour\nhome\nin\nthe\nUS,\n00:00:25.900\nyou\nown\nan\nassetthat\nis\nappreciating\nin\nvalue.\n00:00:29.738\nBuying\nit\nprobably\nrequired\nyouto\nsave\na\nbunch\nof\nmoney,\n00:00:32.406\nand\nalso\nborrow\na\nlot\nof\nmoney,\n00:00:34.200\nwhich\nyou\nneed\nto\npay\nback\nover\ntime;\n00:00:36.036\nas\nopposed\nto\nrenting\

## Load the LLM Model

In [4]:
local_path = (
    "./gpt4all-falcon-q4_0.gguf"  # replace with your desired local file path
)
#model = GPT4All("mistral-7b-instruct-v0.1.Q4_0.gguf")

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
model = GPT4All(model=local_path, callbacks=callbacks, verbose=True)


In [5]:
# Constants: Define constants used in the code.
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100 # control for chunk overlap by denoting how many characters of the previous chunk comes into the next chunk (for continuity)

# Create an instance of RecursiveCharacterTextSplitter with the desired chunk size and overlap
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP, 
    length_function=len  # function used to measure chunk size
)

# split documents into smaller chunks
splits = r_splitter.split_documents(data)


### Ask your question

In [6]:
# Additional context about the file
inital_context="The file contains the youtube transcript of a video."

In [7]:
question = "In the context of the video, what are the zoning laws"
### Summarize in 100 words on what is this video about
#### "Explain promo dampening to a 10 year old Kid"

### Find embeddings of the question

In [8]:

# Use embeddings to combine the long form text into manageable token size
gpt4all_embd = GPT4AllEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

# Find the top 5 documents relevant to the question
docs = vectorstore.similarity_search(question, k=5)

q_emb = gpt4all_embd.embed_query(question)
q_vec = np.array(q_emb)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [9]:
for d in docs:
    emb = gpt4all_embd.embed_query(d.page_content)
    vec = np.array(emb)
    cosine = cosine_similarity(q_vec, vec)
    print(cosine)

0.4669892037973935
0.2834831026012385
0.24342613192268564
0.20228547581888004
0.19476953453152268


In [10]:
for i in docs:
    print(i)

page_content="vacancy\nrate”\n00:03:17.530\nover\nabout\n70\nyears.\n00:03:18.531\nIt's\nthe\npercent\nof\nhomes\nin\nthe\nUS\n00:03:20.533\nthat\nare\nactually\nfor\nsale.\n00:03:22.076\nAnd\nit's\ncurrently\nthe\nlowest\n00:03:23.703\nsince\nwe\nstarted\ncollecting\ndata.\n00:03:25.955\nAnd\nthat\nis\npartly\nthe\nresult\nof\nrestrictive\n“zoning\nlaws”:\n00:03:30.168\nlocally-set\nrules,\nall\nacross\nthe\nUS,\n00:03:32.295\nthat\nregulate\nthingslike\nwhere\nhomes\ncan\nand\ncan't\nbe\nbuilt,\n00:03:36.299\nhow\nmany\nunits\na\nbuilding\ncan\nhave\nin\nit," metadata={'page': 1, 'source': 'median_price_vox.pdf'}
page_content="00:07:18.688\nthe\n3%,\n4%,\n5%\nzone\n--\n00:07:20.565\nit\nmeans\nthat,\nsince\nselling\ntheir\nhome\nmight\nmean\n00:07:22.942\nhaving\nto\nbuy\na\nnew\none\nat\nan\ninterest\nrate\nup\nhere,\n00:07:27.029\nfewer\npeople\nare\nselling\ntheir\nhomes.\n00:07:29.490\nIt\nhelps\nkeep\nthis,\nthat\nhomeowner\nvacancy\nrate,\nlow.\n00:07:32.785\nAnd\nit\nkeeps\nho

In [23]:

# ### Create the Simple template for question and answer
# template = """
# Question: {question}
# Answer: Let's think step by step.
# """

# prompt = PromptTemplate(template=template, input_variables=["question"])

# llm_chain = LLMChain(prompt=prompt, llm=model)


In [11]:
# Define a prompt template. This is a format for the text input we'll give to our model.
# It tells the model how to structure its response and what to do in different situations.
template = """I will provide you pieces of [Context] to answer the [Question]. Provide answer based on the content and context provided. \
If you don't know the answer based on [Context] just say that you don't know, don't try to make up an answer. \
[Context]: {context} \
[Question]: {question} \
Helpful Answer:"""

final_template=inital_context + template

# If your answer includes any sort of list, return it in bullets. \
# Format your answer to Markdown. \

# Create a PromptTemplate object from our string template.
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)



# Create a RetrievalQA object. This uses our language model (llm) and a retriever,
# which is our vector database (vectordb). This object will handle asking our model questions
# and retrieving relevant documents to help answer them.
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


# Ask our question to the qa_chain, and store the result.
result = qa_chain({"query": question})



 The video discusses how restrictive zoning laws can make it difficult for people to buy homes in the US. These laws regulate things like where homes can and cannot be built, which can lead to fewer homes being built and sold on the market. This can result in a higher cost of housing and less availability of affordable options.

In [12]:
from IPython.display import Markdown, display

def print_better_text(text):
    display(Markdown(text))

# Example usage
print_better_text("### Question: "+question+"\n###"+result["result"])

### Question: In the context of the video, what are the zoning laws
### The video discusses how restrictive zoning laws can make it difficult for people to buy homes in the US. These laws regulate things like where homes can and cannot be built, which can lead to fewer homes being built and sold on the market. This can result in a higher cost of housing and less availability of affordable options.